Imports

In [11]:
from preprocess import Preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from ..breast_cancer.evaluation import plot_confusion_matrix, plot_feature_importance, plot_parameter_sensitivity, get_results, get_cross_validation_scores, save_results_to_csv
import pandas as pd
import matplotlib.pylab as plt
from scipy.stats import randint
import numpy as np
import time

ImportError: attempted relative import with no known parent package

## 1. Base Model

In [3]:
pp = Preprocessing()
df = pp.load_preprocessed_speeddating(split = False, scale = False)
X_train, X_test, y_train, y_test = pp.load_preprocessed_speeddating()

In [ ]:
# Get feature and target names
temp_df = df.drop(['ID', 'class'], axis=1)
feature_names = temp_df.columns.tolist()
target_names = ['False (0)', 'True (1)']

# Train a baseline random forest
rf_base = RandomForestClassifier(
    n_estimators=100, 
    random_state=42,
    class_weight= 'balanced',
    oob_score=True 
)
rf_base.fit(X_train, y_train)

y_pred_base = rf_base.predict(X_test)

get_results(y_test, y_pred_base, target_names)